<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#原始的main.py的代码" data-toc-modified-id="原始的main.py的代码-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>原始的main.py的代码</a></span></li><li><span><a href="#make_train_data()——数据读取、封装、分发" data-toc-modified-id="make_train_data()——数据读取、封装、分发-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>make_train_data()——数据读取、封装、分发</a></span><ul class="toc-item"><li><span><a href="#handler()" data-toc-modified-id="handler()-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>handler()</a></span><ul class="toc-item"><li><span><a href="#打印调用CPU的数量" data-toc-modified-id="打印调用CPU的数量-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>打印调用CPU的数量</a></span></li><li><span><a href="#raw_data-=-read_all_raw_data()——所有数据打包成类" data-toc-modified-id="raw_data-=-read_all_raw_data()——所有数据打包成类-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>raw_data = read_all_raw_data()——所有数据打包成类</a></span><ul class="toc-item"><li><span><a href="#DataHolder()" data-toc-modified-id="DataHolder()-2.1.2.1"><span class="toc-item-num">2.1.2.1&nbsp;&nbsp;</span>DataHolder()</a></span><ul class="toc-item"><li><span><a href="#read_raw_data()" data-toc-modified-id="read_raw_data()-2.1.2.1.1"><span class="toc-item-num">2.1.2.1.1&nbsp;&nbsp;</span>read_raw_data()</a></span></li></ul></li></ul></li><li><span><a href="#分发数据以形成test_users、dataset、train_dataset、y_answer" data-toc-modified-id="分发数据以形成test_users、dataset、train_dataset、y_answer-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>分发数据以形成test_users、dataset、train_dataset、y_answer</a></span></li><li><span><a href="#recall_results-=-calc_and_recall(dataset,-train_dataset,-test_users,-articles_dic,-cpu_cores,-offline,-y_answer)" data-toc-modified-id="recall_results-=-calc_and_recall(dataset,-train_dataset,-test_users,-articles_dic,-cpu_cores,-offline,-y_answer)-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>recall_results = calc_and_recall(dataset, train_dataset, test_users, articles_dic, cpu_cores, offline, y_answer)</a></span><ul class="toc-item"><li><span><a href="#sims-=-_calc_sim(dataset,-articles_dic,-cpu_cores,-offline)" data-toc-modified-id="sims-=-_calc_sim(dataset,-articles_dic,-cpu_cores,-offline)-2.1.4.1"><span class="toc-item-num">2.1.4.1&nbsp;&nbsp;</span>sims = _calc_sim(dataset, articles_dic, cpu_cores, offline)</a></span><ul class="toc-item"><li><span><a href="#i2i_30k_sim()" data-toc-modified-id="i2i_30k_sim()-2.1.4.1.1"><span class="toc-item-num">2.1.4.1.1&nbsp;&nbsp;</span>i2i_30k_sim()</a></span><ul class="toc-item"><li><span><a href="#_i2i_30k_sim_core()" data-toc-modified-id="_i2i_30k_sim_core()-2.1.4.1.1.1"><span class="toc-item-num">2.1.4.1.1.1&nbsp;&nbsp;</span>_i2i_30k_sim_core()</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#create_train_data()" data-toc-modified-id="create_train_data()-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>create_train_data()</a></span><ul class="toc-item"><li><span><a href="#ds-=-neg_sampling(ds)" data-toc-modified-id="ds-=-neg_sampling(ds)-2.1.5.1"><span class="toc-item-num">2.1.5.1&nbsp;&nbsp;</span>ds = neg_sampling(ds)</a></span></li><li><span><a href="#get_user_features()" data-toc-modified-id="get_user_features()-2.1.5.2"><span class="toc-item-num">2.1.5.2&nbsp;&nbsp;</span>get_user_features()</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#test()" data-toc-modified-id="test()-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>test()</a></span><ul class="toc-item"><li><span><a href="#prepare_dataset()" data-toc-modified-id="prepare_dataset()-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>prepare_dataset()</a></span></li><li><span><a href="#make_recommend_dict()" data-toc-modified-id="make_recommend_dict()-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>make_recommend_dict()</a></span></li><li><span><a href="#calc_mrr_and_hit()" data-toc-modified-id="calc_mrr_and_hit()-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>calc_mrr_and_hit()</a></span></li></ul></li><li><span><a href="#make_test_data()" data-toc-modified-id="make_test_data()-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>make_test_data()</a></span></li><li><span><a href="#run()" data-toc-modified-id="run()-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>run()</a></span></li></ul></div>

# 原始的main.py的代码

后面是对这个代码的逐句分析，以及所有引用的包、其他.py文件引用的解释

In [ ]:
import pandas as pd
import numpy as np
import sys
from data_holder import DataHolder
from tqdm import tqdm
import pickle
import math
import time
import multiprocessing as mp
import pickle
import lightgbm as lgb
from const import RAW_DATA_FOLDER, OUTPUT_FOLDER, CACHE_FOLDER
from recaller import calc_and_recall
from csv_handler import create_train_data
from sklearn.model_selection import train_test_split

def read_raw_data(filename, cb=None):
    data = pd.read_csv(RAW_DATA_FOLDER + filename)
    return cb(data) if cb is not None else data

# def read_all_raw_data(filenames=['articles.csv', 'train_click_log.csv', 'testB_click_log_Test_B.csv', 'testA_click_log.csv']):
#     return DataHolder(*[read_raw_data(filename) for filename in filenames])
# 我确实没找到testB_click_log_Test_B.csv，所以我删掉了他，然后运行代码
def read_all_raw_data(filenames=['articles.csv', 'train_click_log.csv', 'testA_click_log.csv']):
    return DataHolder(*[read_raw_data(filename) for filename in filenames])


def calc_mrr_and_hit(recommend_dict, y, k=5):
    #assert len(recommend_dict) == len(y)
    sum_mrr = 0.0
    sum_hit = 0.0
    sum_hit_detail = np.repeat(0.0, 5)
    user_cnt = len(recommend_dict.keys())

    for user_id, recommend_items in recommend_dict.items():
        answer = y[user_id] if user_id in y else -1
        if (answer in recommend_items) and (recommend_items.index(answer) < k):
            sum_hit += 1
            sum_mrr += 1 / (recommend_items.index(answer) + 1)
            sum_hit_detail[recommend_items.index(answer)] += 1

    return (sum_mrr / user_cnt), (sum_hit / user_cnt), (sum_hit_detail / user_cnt)

def create_submission(recommend_dict):
    _data = [{'user_id': user_id,
        'article_1': art_id_list[0],
        'article_2': art_id_list[1],
        'article_3': art_id_list[2],
        'article_4': art_id_list[3],
        'article_5': art_id_list[4]} for user_id, art_id_list in tqdm(recommend_dict.items())]
    _t = pd.DataFrame(_data)
    _t.sort_values('user_id', inplace=True)
    _t.to_csv(OUTPUT_FOLDER + 'result.csv', index=False)

def handler(offline=True):
    cpu_cores = mp.cpu_count()
    print('使用CPU核心数: {}'.format(cpu_cores))
    print('开始{}数据验证处理'.format('线下' if offline else '线上'))
    raw_data = read_all_raw_data()
    test_users = raw_data.get_test_users(offline)

    _user_id_list = list(test_users.keys())
    user_id_min = np.min(_user_id_list)
    user_id_max = np.max(_user_id_list)
    print('获得{}用户集合{}件 [{} ~ {}]'.format('验证' if offline else '测试', len(test_users), user_id_min, user_id_max))

    dataset = raw_data.get_item_dt_groupby_user()

    if offline:
        train_dataset, y_answer = raw_data.get_train_dataset_and_answers(test_users)
    else:
        train_dataset = raw_data.get_train_dataset_for_online(test_users)
        y_answer = None

    print('训练数据({}件)'.format(np.sum([len(ts_list) for user_id, ts_list in train_dataset.items()])))

    articles_dic = dict(list(raw_data.get_articles().apply(lambda x: (x['article_id'], dict(x)), axis=1)))
    print('获得文章字典({}件)'.format(len(articles_dic.keys())))

    recall_results = calc_and_recall(dataset, train_dataset, test_users, articles_dic, cpu_cores, offline, y_answer)
    create_train_data(raw_data, train_dataset, test_users, articles_dic, recall_results, offline, y_answer)
    print("(<finished>handler")

def make_train_data():
    # Part_【1.1】_make_train_data
    handler()
    print("(<finished>Part_【1.1】_make_train_data_")

def make_test_data():
    handler(False)
    # Part_【2】make_test_data

def prepare_dataset(df):
    agg_column = [column for column in df.columns if column != 'user_id'][0]
    df.sort_values('user_id', inplace=True)
    grp_info = df.groupby('user_id', as_index=False).count()[agg_column].values
    y = df['answer'] if 'answer' in df.columns else None
    return df.drop(columns=['answer']) if 'answer' in df.columns else df, grp_info, y

def make_recommend_dict(X_val, y_pred):
    X_val['pred'] = y_pred
    _t = X_val.groupby('user_id')\
        .apply(lambda x: list(x.sort_values('pred', ascending=False)['article_id'].head(5)))\
        .reset_index()\
        .rename(columns={0: 'item_list'})

    recommend_dict = dict(zip(_t['user_id'], _t['item_list']))    
    return recommend_dict

def test():
    df_train = pd.read_csv(CACHE_FOLDER + 'train.csv')

    # clf = lgb.LGBMRanker(random_state=777, n_estimators=1000)
    # 调低 n_estimators数量
    clf = lgb.LGBMRanker(random_state=777, n_estimators=100)

    users = df_train['user_id'].unique()
    train_users, _test_users = train_test_split(users, test_size=0.2, random_state=98)
    test_users, val_users = train_test_split(_test_users, test_size=0.5, random_state=38)
    df_new_train = df_train.merge(pd.DataFrame(train_users, columns=['user_id']))
    df_test = df_train.merge(pd.DataFrame(test_users, columns=['user_id']))
    df_val = df_train.merge(pd.DataFrame(val_users, columns=['user_id']))

    X_train, X_grp_train, y_train = prepare_dataset(df_new_train)
    X_test, X_grp_test, y_test = prepare_dataset(df_test)
    X_val, X_grp_val, _ = prepare_dataset(df_val)

    def handle_columns(X):
        return X.drop(columns=['user_id', 'article_id'])

    _X_train = handle_columns(X_train)

    clf.fit(_X_train, y_train, group=X_grp_train, eval_set=[(handle_columns(X_test), y_test)], eval_group=[X_grp_test], eval_at=[1, 2, 3, 4, 5], eval_metric=['ndcg', ], early_stopping_rounds=50, verbose=False)
    print('Best iteration: {}'.format(clf.best_iteration_))


    for X, X_grp, df, title in [(X_test, X_grp_test, df_test, 'Test Set'), (X_val, X_grp_val, df_val, 'Validation Set')]:
        print('[{}]'.format(title))
        y_pred = clf.predict(handle_columns(X), group=X_grp, num_iteration=clf.best_iteration_)
        recommend_dict = make_recommend_dict(X, y_pred)
        answers = dict(df.loc[df['answer'] == 1, ['user_id', 'article_id']].values)
        mrr, hit, details = calc_mrr_and_hit(recommend_dict, answers)
        print('MRR: {} / HIT: {}'.format(mrr, hit))
        print(' / '.join(['%.2f' % detail for detail in details]))

    for column, score in sorted(zip(_X_train.columns, clf.feature_importances_), key=lambda x: x[1], reverse=True):
        print('{}: {}'.format(column, score))
    print(" Part_【1.2】make_test_data")

def run():
    df_train = pd.read_csv(CACHE_FOLDER + 'train.csv')
    df_test = pd.read_csv(CACHE_FOLDER + 'test.csv')

    # clf = lgb.LGBMRanker(random_state=777, n_estimators=1000)
    # !!!!!!!!!建议你将n_estimators变小一些，这样运算速度快!!!!!!!!!!!
    # 调低 n_estimators数量
    clf = lgb.LGBMRanker(random_state=777, n_estimators=100)

    users = df_train['user_id'].unique()
    train_users, eval_users = train_test_split(users, test_size=0.2, random_state=77)
    df_new_train = df_train.merge(pd.DataFrame(train_users, columns=['user_id']))
    df_eval = df_train.merge(pd.DataFrame(eval_users, columns=['user_id']))

    X_train, X_grp_train, y_train = prepare_dataset(df_new_train)
    X_eval, X_grp_eval, y_eval = prepare_dataset(df_eval)
    X_test, X_grp_test, _ = prepare_dataset(df_test)

    def handle_columns(X):
        return X.drop(columns=['user_id', 'article_id'])

    _X_train = handle_columns(X_train)

    clf.fit(_X_train, y_train, group=X_grp_train, eval_set=[(handle_columns(X_eval), y_eval)], eval_group=[X_grp_eval], eval_at=[1, 2, 3, 4, 5], eval_metric=['ndcg', ], early_stopping_rounds=50, verbose=False)
    print('Best iteration: {}'.format(clf.best_iteration_))
    y_pred = clf.predict(handle_columns(X_test), group=X_grp_test, num_iteration=clf.best_iteration_)
    
    for column, score in sorted(zip(_X_train.columns, clf.feature_importances_), key=lambda x: x[1], reverse=True):
        print('{}: {}'.format(column, score))

    recommend_dict = make_recommend_dict(X_test, y_pred)

    create_submission(recommend_dict)
    print("<finished>Part_【3】run")

if __name__ == "__main__":
    print("<Start># Part_【1】_make_train_data")
    make_train_data()
    print("<Start># Part_【1.2】make_test_data")
    test()
    print("<Start># Part_【2】make_test_data")
    make_test_data()
    print("<Start># Part_【3】run")
    run()



# make_train_data()——数据读取、封装、分发

In [ ]:
print("<Start># Part_【1】_make_train_data");print()
make_train_data()

In [ ]:
def make_train_data():
    # Part_【1.1】_make_train_data
    handler()
    print("(<finished>Part_【1.1】_make_train_data_")

## handler()

In [ ]:
def handler(offline=True):
    
#     2.1.1  打印调用CPU的数量
    # 数一下一共多少个cpu被使用了
    cpu_cores = mp.cpu_count()
    print('使用CPU核心数: {}'.format(cpu_cores))
    print('开始{}数据验证处理'.format('线下' if offline else '线上'))
    
    # 2.1.1  read_all_raw_data()
    raw_data = read_all_raw_data()
    test_users = raw_data.get_test_users(offline) # ？？？？这句话不懂了，没头没尾，为什raw_data有get_test_users的方法？
    
    _user_id_list = list(test_users.keys())
    user_id_min = np.min(_user_id_list)
    user_id_max = np.max(_user_id_list)
    # 线下就说“验证”，线上就说“测试”，说明test_users的数量，id是从几到几
    print('获得{}用户集合{}件 [{} ~ {}]'.format('验证' if offline else '测试', len(test_users), user_id_min, user_id_max))

    dataset = raw_data.get_item_dt_groupby_user()
    
    # 将所有数据文件封装起来的raw_data里面的数据，分发给各个变量
    if offline:
        train_dataset, y_answer = raw_data.get_train_dataset_and_answers(test_users)
    else:
        train_dataset = raw_data.get_train_dataset_for_online(test_users)
        y_answer = None

    print('训练数据({}件)'.format(np.sum([len(ts_list) for user_id, ts_list in train_dataset.items()])))

    articles_dic = dict(list(raw_data.get_articles().apply(lambda x: (x['article_id'], dict(x)), axis=1)))
    print('获得文章字典({}件)'.format(len(articles_dic.keys())))

    
#     1.1.2 calc_and_recall
    recall_results = calc_and_recall(dataset, train_dataset, test_users, articles_dic, cpu_cores, offline, y_answer)
    
    
    create_train_data(raw_data, train_dataset, test_users, articles_dic, recall_results, offline, y_answer)
    print("(<finished>handler")

### 打印调用CPU的数量

In [ ]:
    # 数一下一共多少个cpu被使用了
    cpu_cores = mp.cpu_count()
    print('使用CPU核心数: {}'.format(cpu_cores))
    print('开始{}数据验证处理'.format('线下' if offline else '线上'))
    

###  raw_data = read_all_raw_data()——所有数据打包成类

In [ ]:
# 'testB_click_log_Test_B.csv' 这个文件是我问王百万学长要的，网上没有提供
# 你删除掉这个文件并不影响程序的运行
# 你加上了反而报错——key error,应该是数据的维度不对————暂时删掉了，后面去解决这个错误

# def read_all_raw_data(filenames=['articles.csv', 'train_click_log.csv', 'testB_click_log_Test_B.csv', 'testA_click_log.csv']):
#     return DataHolder(*[read_raw_data(filename) for filename in filenames])

def read_all_raw_data(filenames=['articles.csv', 'train_click_log.csv', 'testA_click_log.csv']):
    return DataHolder(*[read_raw_data(filename) for filename in filenames])

#### DataHolder()

In [ ]:
# 这里注意这个DatsHolder是下面这句话引入的。这个类里面就是有get_test_users（）和get_item_dt_groupby_user（），get_train_dataset_and_answers（）的方法，可以生成所需的数据

In [5]:
# DataHolder这个类的定义信息如下

In [ ]:
class DataHolder:
    # 我 实际调用向内传入['articles.csv', 'train_click_log.csv', 'testA_click_log.csv']
        # articles代表'articles.csv'：文章的基本信息（文章的id、类别、创建时间、字数）

        # train_click_log代表"train_click_log.csv":
        # test_click_log代表"testA_click_log.csv"
        # trainB_click_log代表"testB_click_log_Test_B.csv"(可以不填，不填就是None)
            # 上面这三组数据，维度都是一样的，每行代表一个用户的一次点击
            # 每一列分别是，用户的id,点击的文章的代号id,点击的时间，以及每次点击相关的数据（文章的类别，什么平台点击的，什么设备点击，等等很多列）
    def __init__(self, articles, train_click_log, test_click_log, trainB_click_log=None):
        # 【1】将输入的数据分发出去
        self.articles = articles
        self.train_click_log = train_click_log
        self.test_click_log = test_click_log
        self.trainB_click_log = trainB_click_log
            # all_click_log表示的意思是，
                # 如果trainB_click_log非空，就按照（self.train_click_log；self.trainB_click_log；self.test_click_log）3个做一个数据集合并
                # 如果 没有trainB_click_log，就只合并（self.train_click_log；self.test_click_log）2个
        self.all_click_log = self.train_click_log.append(self.trainB_click_log).append(self.test_click_log) if self.trainB_click_log is not None else self.train_click_log.append(self.test_click_log)
                # 将合并的数据集里面，user_id，文章_id,点击时间有重复的删除————删除一些冗余的信息
        self.all_click_log.drop_duplicates(['user_id', 'click_article_id', 'click_timestamp'], inplace=True)

        # 将读取的这些数据的基本信息打印出来
        print('从train_click_log读取{}件(UserId=[{},{}])'.format(len(self.train_click_log), self.train_click_log['user_id'].min(), self.train_click_log['user_id'].max()))
        print('从test_click_log读取{}件(UserId=[{},{}])'.format(len(self.test_click_log), self.test_click_log['user_id'].min(), self.test_click_log['user_id'].max()))
        if self.trainB_click_log is not None:
            print('从trainB_click_log读取{}件(UserId=[{},{}])'.format(len(self.trainB_click_log), self.trainB_click_log['user_id'].min(), self.trainB_click_log['user_id'].max()))
        print('使用训练集all_click_log共{}件(UserId=[{},{}])'.format(len(self.all_click_log), self.all_click_log['user_id'].min(), self.all_click_log['user_id'].max()))


        # 【2.1】 将你合并好的所有数据DataFrame，存储成dataset.pkl文件，转为字典格式命名为self.dataset
            # 存储成.pkl文件以后，你第二次不必再做一次数据合并和转dict了
            # 字典应该是运算速度比dataframe更快，所以选择这种数据类型
        filename = 'dataset.pkl'
        if isfile(CACHE_FOLDER + filename):
            print('直接从文件{}中读取dataset'.format(filename))
                # "rb"就是reading模式，"wb"是write模式
            self.dataset = pickle.load(open(CACHE_FOLDER + filename, 'rb'))
        else:
            start_time = time.time()
            #
            _t = self.all_click_log.sort_values('click_timestamp').groupby('user_id')\
                .apply(lambda x: list(zip(x['click_article_id'], x['click_timestamp'])))\
                .reset_index()\
                .rename(columns={0: 'item_dt_list'})
            self.dataset = dict(zip(_t['user_id'], _t['item_dt_list']))
            print('dataset对象完毕({}秒)'.format('%.2f' % (time.time() - start_time)))
            print('保存dataset至文件{}中'.format(filename))
            pickle.dump(self.dataset, open(CACHE_FOLDER + filename, 'wb'))

        # 【2.2】 将你合并好的所有数据DataFrame，存储成train_users_dic.pkl文件，转为字典格式命名为self.train_users_dic
            # 字典内容为(user_id, timestamp)，字典用于供训练用
        filename = 'train_users_dic.pkl'
        if isfile(CACHE_FOLDER + filename):
            print('直接从文件{}中读取train_users_dic'.format(filename))
            self.train_users_dic = pickle.load(open(CACHE_FOLDER + filename, 'rb'))
        else:
            start_time = time.time()
            self.train_users_dic = {}
                # 这里这个tqdm，代表的意思就是进度条那个东西
            for user_id, items in tqdm(self.dataset.items()):
                ts_list = pd.Series([item[1] for item in items])
                    # 向train_users_dic里面添加key为user_id，value为后面的东西
                self.train_users_dic[user_id] = list(ts_list.loc[ts_list.shift(-1) - ts_list == 30000])
            print('train_users_dic对象完毕({}秒)'.format('%.2f' % (time.time() - start_time)))
            print('保存train_users_dic至文件{}中'.format(filename))
            pickle.dump(self.train_users_dic, open(CACHE_FOLDER + filename, 'wb'))


    # 【3】写好调用接口，后面分发数据，就用下面的
    def get_articles(self):
        return self.articles
    def get_train_click_log(self):
        return self.train_click_log
    def get_test_click_log(self):
        return self.test_click_log
    def get_all_click_log(self):
        return self.all_click_log

    def get_user_list(self):
        # 返回用户的id
        return self.train_click_log['user_id'].unique()
    def get_item_dt_groupby_user(self):
        return self.dataset


    def users_df2dic(self, df_users):
        _t = df_users.sort_values('click_timestamp').groupby('user_id')\
            .apply(lambda x: set(x['click_timestamp']))\
            .reset_index()\
            .rename(columns={0: 'ts_set'})
        return dict(zip(_t['user_id'], _t['ts_set']))

    # 生成测试所需的数据
        # 详细解释：应该是用来将过去的数据（self.train_users_dic），进行格式处理，然后随机取样
    def get_test_users(self, offline, samples=100000):
        if offline:
            # 一维数组化
            users = []
            for user_id, ts_list in self.train_users_dic.items():
                # for ts in ts_list:
                #     users.append((user_id, ts))
                if len(ts_list) > 0:
                    users.append((user_id, ts_list[-1]))
            np.random.seed(42)
            idx_list = np.random.choice(len(users), samples, replace=False)
            selected_users = [users[idx] for idx in idx_list]
            # 字典化
            return self.users_df2dic(pd.DataFrame(selected_users, columns=['user_id', 'click_timestamp']))
        else:
            return self.users_df2dic(self.test_click_log.groupby('user_id').max('click_timestamp').reset_index()[['user_id', 'click_timestamp']])

    def take_last(self, items, last=1):
        if len(items) <= last:
            return items.copy(), items[0]
        else:
            return items[:-last], items[-last]

    # 用来将dataset和预测值（y_answer)的混合字典，拆分成两个字典
    def get_train_dataset_and_answers(self, test_users):
        start_time = time.time()
        train_dataset = {}
        y_answer = {}

            # tqdm，代表进度条
        for user_id, ts_set in tqdm(test_users.items()):
            items = self.dataset[user_id]
            for last_clicked_timestamp in ts_set:
                idx = [item[1] for item in items].index(last_clicked_timestamp)
                train_dataset.setdefault(user_id, {})
                train_dataset[user_id][last_clicked_timestamp] = items[0:idx+1]
                y_answer.setdefault(user_id, {})
                y_answer[user_id][last_clicked_timestamp] = items[idx+1][0]
        print('训练集和答案分割完毕({}秒)'.format('%.2f' % (time.time() - start_time)))
        return train_dataset, y_answer

    def get_train_dataset_for_online(self, test_users):
        start_time = time.time()
        train_dataset = {}

        for user_id, ts_set in tqdm(test_users.items()):
            items = self.dataset[user_id]
            for last_clicked_timestamp in ts_set:
                train_dataset.setdefault(user_id, {})
                train_dataset[user_id][last_clicked_timestamp] = items
        print('测试集制作完毕({}秒)'.format('%.2f' % (time.time() - start_time)))
        return train_dataset


##### read_raw_data()

In [ ]:
def read_raw_data(filename, cb=None):
    data = pd.read_csv(RAW_DATA_FOLDER + filename)
    return cb(data) if cb is not None else data

In [ ]:
from const import RAW_DATA_FOLDER, OUTPUT_FOLDER, CACHE_FOLDER

In [ ]:
# const.py这个文件中这样写
# # RAW_DATA_FOLDER = '../tcdata/'
# # OUTPUT_FOLDER = '../prediction_result/'
# # CACHE_FOLDER = '../user_data/'

上面是读取本文件上一一层文件夹下的tcdata内的这些文件

就是把['articles.csv', 'train_click_log.csv', 'testB_click_log_Test_B.csv', 'testA_click_log.csv']这些文件夹逐一读取

### 分发数据以形成test_users、dataset、train_dataset、y_answer
调用类的实例（raw_data）方法，以实现数据分发

test_users = raw_data.get_test_users(offline)

dataset = raw_data.get_item_dt_groupby_user()——字典，用户id:[用户所有的点击数据]（待定？？）

train_dataset, y_answer = raw_data.get_train_dataset_and_answers(test_users)

In [ ]:
    test_users = raw_data.get_test_users(offline)
    _user_id_list = list(test_users.keys())
    user_id_min = np.min(_user_id_list)
    user_id_max = np.max(_user_id_list)
    print('获得{}用户集合{}件 [{} ~ {}]'.format('验证' if offline else '测试', len(test_users), user_id_min, user_id_max))
    dataset = raw_data.get_item_dt_groupby_user()
    if offline:
        train_dataset, y_answer = raw_data.get_train_dataset_and_answers(test_users)
    else:
        train_dataset = raw_data.get_train_dataset_for_online(test_users)
        y_answer = None
    print('训练数据({}件)'.format(np.sum([len(ts_list) for user_id, ts_list in train_dataset.items()])))
    articles_dic = dict(list(raw_data.get_articles().apply(lambda x: (x['article_id'], dict(x)), axis=1)))
    print('获得文章字典({}件)'.format(len(articles_dic.keys())))

### recall_results = calc_and_recall(dataset, train_dataset, test_users, articles_dic, cpu_cores, offline, y_answer)

In [ ]:
recall_results = calc_and_recall(dataset, train_dataset, test_users, articles_dic, cpu_cores, offline, y_answer)

In [ ]:
# calc_and_recall 是通过import进来的，来自recaller.py

# from recaller import calc_and_recall

calc_and_recall的输入变量都代表什么意思呢？

In [ ]:
# def calc_and_recall(dataset, train_dataset, test_users, articles_dic, cpu_cores, offline, answers=None):
#     # **这些输入变量分别代表什么实际的意思？**
#     # dataset——字典，用户id: [用户所有的点击数据]（待定？？）
#         # （main.py->data_holder.py)dataset = raw_data.get_item_dt_groupby_user()
#     # train_dataset_，应该是代表各种各样的用户信息
#         # （main.py->data_holder.py)train_dataset, y_answer = raw_data.get_train_dataset_and_answers(test_users)
#     # test_users——字典，从self.train_users_dic中随机抽取sampels个用户的信息（待定？？）
#         # （main.py->data_holder.py) test_users = raw_data.get_test_users(offline)
#     # articles_dic——'articles.csv'的内容-：文章的基本信息（文章的id、类别、创建时间、字数）
#         # 文章id:文章所有的信息（待定？？）
#         # （main.py->data_holder.py) articles_dic = dict(list(raw_data.get_articles().apply(lambda x: (x['article_id'], dict(x)), axis=1)))

#### sims = _calc_sim(dataset, articles_dic, cpu_cores, offline)

 _calc_sim 这个函数就定义在recallr.py这个文件内

_calc_sim中的输入的变量代表什么意思？

In [ ]:
# def _calc_sim(dataset, articles_dic, cpu_cores, offline):
#     # ***输入变量代表什么意思？***
#     # dataset——字典，用户id: [用户所有的点击数据]（待定？？）
#         # （main.py->data_holder.py)dataset = raw_data.get_item_dt_groupby_user()
#     # articles_dic——'articles.csv'的内容-：文章的基本信息（文章的id、类别、创建时间、字数）
#         # 文章id:文章所有的信息（待定？？）
#         # （main.py->data_holder.py) articles_dic = dict(list(raw_data.get_articles().apply(lambda x: (x['article_id']

##### i2i_30k_sim()

调用这个函数的代码

In [ ]:
#     # 【2.1.4.1.1】  i2i_30k_sim
#     # 计算各种相似度
#     num = len([i2i_30k_sim])

#     start_time = time.time()
#     print('召回前的计算处理开始({}件)'.format(num))

#     sims = {}
#     sims['i2i_30k_sim'] = i2i_30k_sim(dataset, cpu_cores, offline)

#     print('召回前的计算处理结束({}秒)'.format('%.2f' % (time.time() - start_time)))

#     return sims

这个函数的完整的代码如下：

In [ ]:
# def i2i_30k_sim(dataset, n_cpu, offline, max_related=50):
#     # ***传入的变量代表的意思是什么？***
#     # dataset——字典，用户id: [用户所有的点击数据]（待定？？）
#         # （main.py->data_holder.py)dataset = raw_data.get_item_dt_groupby_user()

#     # 填充进去的文字是在大括号那个位置
#     filename = 'i2i_30k_sim_{}.pkl'.format('offline' if offline else 'online')
#     # 如果这个地址（'../user_data/'）有上面那个字符串那个样子的pkl模型文件，就直接返回这个模型的信息，
#     #   之前计算过，就不必重复计算了，读取之前算好的就行了
#     if isfile(CACHE_FOLDER + filename):
#         print('直接从文件{}中读取计算好的i2i_30k相似度'.format(filename))
#         return pickle.load(open(CACHE_FOLDER + filename, 'rb'))

#     # 计算相似度
#     start_time = time.time()
#     print('开始计算i2i_30k相似度')
#     i2i_sim_3k = {}
#         # 分配到每一个cpu上有多少个用户的点击数据
#     n_block = (len(dataset.keys()) - 1) // n_cpu + 1
#     keys = list(dataset.keys())
#         # 这个可以让你并行使用多个cpu核
#     pool = mp.Pool(processes=n_cpu)
#         # 进程任务拆分。给每个cpu指定了任务。（不太懂？？）
#     results = [pool.apply_async(_i2i_30k_sim_core, args=(i, keys[i * n_block:(i + 1) * n_block], dataset)) for i in range(0, n_cpu)]
#     pool.close()
#     pool.join()

#     for result in results:
#         _i2i_sim_3k = result.get()

#         for art_id, related_art_id_dic in _i2i_sim_3k.items():
#             i2i_sim_3k.setdefault(art_id, {})
#             for related_art_id, value in related_art_id_dic.items():
#                 i2i_sim_3k[art_id].setdefault(related_art_id, 0)
#                 i2i_sim_3k[art_id][related_art_id] += value

#     print('逆序排序')
#     for art_id, related_arts in tqdm(i2i_sim_3k.items()):
#         sorted_and_topK = sorted(related_arts.items(), key=lambda x: x[1], reverse=True)
#         i2i_sim_3k[art_id] = {
#             'sorted_keys': [art_id for art_id, _ in sorted_and_topK],
#             'related_arts': dict(sorted_and_topK)
#         } 

#     print('i2i_30k相似度计算完毕({}秒)'.format('%.2f' % (time.time() - start_time)))
#     print('保存i2i_30k相似度数据至文件{}中'.format(filename))
#     pickle.dump(i2i_sim_3k, open(CACHE_FOLDER + filename, 'wb'))
#     return i2i_sim_3k


具体解释如下

In [ ]:
# # 填充进去的文字是在大括号那个位置
#     filename = 'i2i_30k_sim_{}.pkl'.format('offline' if offline else 'online')
#     # 如果这个地址（'../user_data/'）有上面那个字符串那个样子的pkl模型文件，就直接返回这个模型的信息，
#     #   之前计算过，就不必重复计算了，读取之前算好的就行了
#     if isfile(CACHE_FOLDER + filename):
#         print('直接从文件{}中读取计算好的i2i_30k相似度'.format(filename))
#         return pickle.load(open(CACHE_FOLDER + filename, 'rb'))

#     # 计算相似度
#     start_time = time.time()
#     print('开始计算i2i_30k相似度')
#     i2i_sim_3k = {}
#         # 分配到每一个cpu上有多少个用户的点击数据
#     n_block = (len(dataset.keys()) - 1) // n_cpu + 1
#     keys = list(dataset.keys())
#         # 这个可以让你并行使用多个cpu核
#     pool = mp.Pool(processes=n_cpu)
#         # 进程任务拆分。给每个cpu指定了任务。（不太懂？？）

###### _i2i_30k_sim_core()

调用这个函数的句子

In [ ]:
results = [pool.apply_async(_i2i_30k_sim_core, args=(i, keys[i * n_block:(i + 1) * n_block], dataset)) for i in range(0, n_cpu)]
    pool.close()

这个函数定义的代码

In [ ]:
# def _i2i_30k_sim_core(job_id, user_id_list, dataset):
#     # ***传进去每个变量代表什么意思***
#     # 一共传进去3个变量，i 、若干个keys、dataset
#     # i：代表使用的是第几个cpu
#     # 所有这些用户的id(keys)分配给每个cpu的id是哪些
#     # dataset——字典，用户id: [用户所有的点击数据]（待定？？
#     _item_counts_dic = {}
#     _i2i_30k_sim = {}

#     start_time = time.time()
#     # 从所有id的这个用户id里，把每个id逐一取出来
#     for user_id in user_id_list:
#         # 从dataset中将每个用户对应的的数据（device类型，所处位置，文章类型等等）提取出来
#         item_dt = dataset[user_id]
#         # 所有用户的数据拆分成两部分，抛弃第一部分，保留第二部分
#         ts_list = pd.Series([ts for _, ts in item_dt])
#         # 不太懂，应该就是提取了一部分吧，这里30000的原因是“训练集每个用户最后两次点击差值皆为30000”
#         idx_list = [idx for idx, val in dict(ts_list - ts_list.shift(1) == 30000).items() if val]

#         for idx in idx_list:
#             i_art_id, _ = item_dt[idx]
#             j_art_id, _ = item_dt[idx - 1]

#             _i2i_30k_sim.setdefault(i_art_id, {})
#             _i2i_30k_sim[i_art_id].setdefault(j_art_id, 0)
#             _i2i_30k_sim[i_art_id][j_art_id] += 1

#             _i2i_30k_sim.setdefault(j_art_id, {})
#             _i2i_30k_sim[j_art_id].setdefault(i_art_id, 0)
#             _i2i_30k_sim[j_art_id][i_art_id] += 1

#     print('子任务[{}]: 完成i2i_30k相似度的计算。({}秒)'.format(job_id, '%.2f' % (time.time() - start_time)))

#     return _i2i_30k_sim

以下代码，太复杂，先不看，完全不懂

In [ ]:
#  pool.close()
#     pool.join()

#     for result in results:
#         # mp.Pool(processes=n_cpu) 这个类有get方法
#         _i2i_sim_3k = result.get()

#         for art_id, related_art_id_dic in _i2i_sim_3k.items():
#             i2i_sim_3k.setdefault(art_id, {})
#             for related_art_id, value in related_art_id_dic.items():
#                 i2i_sim_3k[art_id].setdefault(related_art_id, 0)
#                 i2i_sim_3k[art_id][related_art_id] += value

#     print('逆序排序')
#     for art_id, related_arts in tqdm(i2i_sim_3k.items()):
#         sorted_and_topK = sorted(related_arts.items(), key=lambda x: x[1], reverse=True)
#         i2i_sim_3k[art_id] = {
#             'sorted_keys': [art_id for art_id, _ in sorted_and_topK],
#             'related_arts': dict(sorted_and_topK)
#         } 

#     print('i2i_30k相似度计算完毕({}秒)'.format('%.2f' % (time.time() - start_time)))
#     print('保存i2i_30k相似度数据至文件{}中'.format(filename))
#     pickle.dump(i2i_sim_3k, open(CACHE_FOLDER + filename, 'wb'))
#     return i2i_sim_3k

### create_train_data()

调用这个函数的代码

In [ ]:
# create_train_data(raw_data, train_dataset, test_users, articles_dic, recall_results, offline, y_answer)

这个函数（create_train_data）是从csv_handler.py中引入进来的

In [8]:
# from csv_handler import create_train_data

这个函数的定义代码如下

In [ ]:
# def create_train_data(raw_data, train_dataset, test_users, articles_dic, recall_results, offline, y_answer):
#     start_time = time.time()
#     keys_ds = []

#     for user_id, ts_set in test_users.items():
#         for last_clicked_timestamp in ts_set:
#             items = np.concatenate([result[user_id][last_clicked_timestamp] for _, result in recall_results.items()])
#             keys_ds.append(list(zip(np.repeat(user_id, len(items)), np.repeat(last_clicked_timestamp, len(items)), items)))

#     ds = pd.DataFrame(np.concatenate(keys_ds), columns=['user_id', 'last_clicked_timestamp', 'article_id'], dtype=np.int64).drop_duplicates()

#     if offline:
#         answer_keys_ds = []
#         # 拼接正确答案标签
#         for user_id, ts_list in y_answer.items():
#             for last_clicked_timestamp, art_id in ts_list.items():
#                 answer_keys_ds.append((user_id, last_clicked_timestamp, art_id))

#         answers = pd.DataFrame(answer_keys_ds, columns=['user_id', 'last_clicked_timestamp', 'article_id'], dtype=np.int64)
#         # 将正确答案融合进数据集
#         answers['answer'] = 1
#         ds = ds.merge(answers, how='left').fillna({'answer': 0})
#         ds['answer'] = ds['answer'].astype(np.int8)

#         # 负采样
#         ds = neg_sampling(ds)

#     ds = ds.merge(raw_data.get_articles()).merge(get_user_features(raw_data, train_dataset, test_users, articles_dic))

#     # 新特征
#     ds['lag_period_last_article'] = ds['last_clicked_timestamp'] - ds['created_at_ts']
#     ds['diff_words_last_article'] = ds['avg_words_count'] - ds['words_count']
#     ds.to_csv(CACHE_FOLDER + '{}.csv'.format('train' if offline else 'test'), index=False)
#     print('{}用的csv文件生成完毕({}秒, {}件)'.format('训练' if offline else '测试', '%.2f' % (time.time() - start_time), 

具体解释如下

<font color=red> 实话说这段代码一句也没看懂，应该逐一运行把每个变量print出来，看看每个变量代表什么意思

In [ ]:
#     start_time = time.time()
#     keys_ds = []

#     for user_id, ts_set in test_users.items():
#         for last_clicked_timestamp in ts_set:
#             items = np.concatenate([result[user_id][last_clicked_timestamp] for _, result in recall_results.items()])
#             keys_ds.append(list(zip(np.repeat(user_id, len(items)), np.repeat(last_clicked_timestamp, len(items)), items)))

#     ds = pd.DataFrame(np.concatenate(keys_ds), columns=['user_id', 'last_clicked_timestamp', 'article_id'], dtype=np.int64).drop_duplicates()

#     if offline:
#         answer_keys_ds = []
#         # 拼接正确答案标签
#         for user_id, ts_list in y_answer.items():
#             for last_clicked_timestamp, art_id in ts_list.items():
#                 answer_keys_ds.append((user_id, last_clicked_timestamp, art_id))

#         answers = pd.DataFrame(answer_keys_ds, columns=['user_id', 'last_clicked_timestamp', 'article_id'], dtype=np.int64)
#         # 将正确答案融合进数据集
#         answers['answer'] = 1
#         ds = ds.merge(answers, how='left').fillna({'answer': 0})
#         ds['answer'] = ds['answer'].astype(np.int8)

#### ds = neg_sampling(ds)

调用这个函数的句子

In [ ]:
# ds = neg_sampling(ds)

这个函数定义在csv_handler.py里面，原始代码如下

In [ ]:
# def neg_sampling(ds, min=1, max=5):
#     start_time = time.time()
#     pos_ds = ds.loc[ds['answer'] == 1]
#     neg_ds = ds.loc[ds['answer'] == 0]

#     def _neg_sampling_func(x):
#         n_sampling = len(x)
#         n_sampling = min if n_sampling < min else (max if n_sampling > max else n_sampling)
#         return x.sample(n=n_sampling, replace=False)

#     neg_ds = pd.concat([
#         neg_ds.groupby(['user_id', 'last_clicked_timestamp']).apply(_neg_sampling_func),
#         neg_ds.groupby('article_id').apply(_neg_sampling_func),
#         ]).drop_duplicates()

#     ret = pd.concat([pos_ds, neg_ds]).reset_index(drop=True)
#     print('负采样处理完毕({}秒, {}->{}件)'.format('%.2f' % (time.time() - start_time), len(ds), len(ret)))
#     return ret

#### get_user_features()

用这个函数的地方

In [ ]:
# ds = ds.merge(raw_data.get_articles()).merge(get_user_features(raw_data, train_dataset, test_users, articles_dic))

这个函数的定义代码

【问：】get_user_features()的输出是什么？

In [ ]:
# def get_user_features(raw_data, train_dataset, test_users, articles_dic):
#     def calc_avg_words_count(items):
#         return np.average([articles_dic[item[0]]['words_count'] for item in items])

#     def calc_min_words_count(items):
#         return np.min([articles_dic[item[0]]['words_count'] for item in items])

#     def calc_max_words_count(items):
#         return np.max([articles_dic[item[0]]['words_count'] for item in items])

#     def calc_lag_between_created_at_ts_and_clicked_ts(items, articles_dic):
#         item = items[-1]
#         return (item[1] - articles_dic[item[0]]['created_at_ts']) / (1000 * 60 * 60 * 24)

#     def calc_lag_between_two_click(items):
#         if len(items) > 1:
#             return (items[-1][1] - items[-2][1]) / (1000 * 60 * 60 * 24)
#         else:
#             return np.nan

#     def calc_lag_between_two_articles(items, articles_dic):
#         if len(items) > 1:
#             return (articles_dic[items[-1][0]]['created_at_ts'] - articles_dic[items[-2][0]]['created_at_ts']) / (1000 * 60 * 60 * 24)
#         else:
#             return np.nan

#     df_users = pd.DataFrame(list(test_users.keys()), columns=['user_id'])

#     # 计算
#     # 1. 用户看新闻的平均字数
#     _data = []
#     for user_id, ts_set in tqdm(test_users.items()):
#         for last_clicked_timestamp in ts_set:
#             _data.append((
#                 user_id,
#                 last_clicked_timestamp,
#                 calc_avg_words_count(train_dataset[user_id][last_clicked_timestamp]),
#                 calc_min_words_count(train_dataset[user_id][last_clicked_timestamp]),
#                 calc_max_words_count(train_dataset[user_id][last_clicked_timestamp]),
#                 calc_lag_between_created_at_ts_and_clicked_ts(train_dataset[user_id][last_clicked_timestamp], articles_dic),
#                 calc_lag_between_two_click(train_dataset[user_id][last_clicked_timestamp]),
#                 calc_lag_between_two_articles(train_dataset[user_id][last_clicked_timestamp], articles_dic),
#                 ))

#     df1 = pd.DataFrame(_data, columns=['user_id', 'last_clicked_timestamp', 'avg_words_count', 'min_words_count', 'max_words_count', 'lag_between_created_at_ts_and_clicked_ts', 'lag_between_two_click', 'lag_between_two_articles'])

#     # 计算用户使用设备，环境等的众数
#     columns = ['user_id','click_environment','click_deviceGroup','click_os','click_country','click_region','click_referrer_type']
#     df2 = df_users.merge(raw_data.get_all_click_log())[columns].groupby('user_id').agg(lambda x: x.value_counts().index[0]).reset_index()

#     return df1.merge(df2)

下面就是其他的一些代码了

In [ ]:
# # 构建新的features，提供更多的预测的原料
#         # 距离上次点击的时差
#     ds['lag_period_last_article'] = ds['last_clicked_timestamp'] - ds['created_at_ts']
#         # 和平均字数之间的差距
#     ds['diff_words_last_article'] = ds['avg_words_count'] - ds['words_count']

#     # 最终生成的数据集有这么多列：
#         # user_id	last_clicked_timestamp	article_id	category_id	created_at_ts	words_count	avg_words_count	min_words_count	max_words_count	lag_between_created_at_ts_and_clicked_ts	lag_between_two_click	lag_between_two_articles	click_environment	click_deviceGroup	click_os	click_country	click_region	click_referrer_type	lag_period_last_article	diff_words_last_article
#     ds.to_csv(CACHE_FOLDER + '{}.csv'.format('train' if offline else 'test'), index=False)
#     print('{}用的csv文件生成完毕({}秒, {}件)'.format('训练' if offline else '测试', '%.2f' % (time.time() - start_

# test()

调用test()的位置

In [ ]:
# print("<Start># Part_【1.2】make_test_data");print()
# test()

test()函数定义的代码

In [ ]:
# def test():
#     df_train = pd.read_csv(CACHE_FOLDER + 'train.csv')

#     # clf = lgb.LGBMRanker(random_state=777, n_estimators=1000)
#     # 调低 n_estimators数量
#     clf = lgb.LGBMRanker(random_state=777, n_estimators=100)

#     users = df_train['user_id'].unique()
#     train_users, _test_users = train_test_split(users, test_size=0.2, random_state=98)
#     test_users, val_users = train_test_split(_test_users, test_size=0.5, random_state=38)
#     df_new_train = df_train.merge(pd.DataFrame(train_users, columns=['user_id']))
#     df_test = df_train.merge(pd.DataFrame(test_users, columns=['user_id']))
#     df_val = df_train.merge(pd.DataFrame(val_users, columns=['user_id']))

#     X_train, X_grp_train, y_train = prepare_dataset(df_new_train)
#     X_test, X_grp_test, y_test = prepare_dataset(df_test)
#     X_val, X_grp_val, _ = prepare_dataset(df_val)

#     def handle_columns(X):
#         return X.drop(columns=['user_id', 'article_id'])

#     _X_train = handle_columns(X_train)

#     clf.fit(_X_train, y_train, group=X_grp_train, eval_set=[(handle_columns(X_test), y_test)], eval_group=[X_grp_test], eval_at=[1, 2, 3, 4, 5], eval_metric=['ndcg', ], early_stopping_rounds=50, verbose=False)
#     print('Best iteration: {}'.format(clf.best_iteration_))


#     for X, X_grp, df, title in [(X_test, X_grp_test, df_test, 'Test Set'), (X_val, X_grp_val, df_val, 'Validation Set')]:
#         print('[{}]'.format(title))
#         y_pred = clf.predict(handle_columns(X), group=X_grp, num_iteration=clf.best_iteration_)
#         recommend_dict = make_recommend_dict(X, y_pred)
#         answers = dict(df.loc[df['answer'] == 1, ['user_id', 'article_id']].values)
#         mrr, hit, details = calc_mrr_and_hit(recommend_dict, answers)
#         print('MRR: {} / HIT: {}'.format(mrr, hit))
#         print(' / '.join(['%.2f' % detail for detail in details]))

#     for column, score in sorted(zip(_X_train.columns, clf.feature_importances_), key=lambda x: x[1], reverse=True):
#         print('{}: {}'.format(column, score))
#     print(" Part_【1.2】make_test_data")

## prepare_dataset()

调用这个函数的地方

In [ ]:
#     X_train, X_grp_train, y_train = prepare_dataset(df_new_train)
#     X_test, X_grp_test, y_test = prepare_dataset(df_test)
#     X_val, X_grp_val, _ = prepare_dataset(df_val)

这个函数是如何定义的

In [ ]:
# def prepare_dataset(df):
#     # 不知道意思是什么？
#     agg_column = [column for column in df.columns if column != 'user_id'][0]
#     # id 按照升序或者降序排列
#     df.sort_values('user_id', inplace=True)
#     # 不太懂
#     grp_info = df.groupby('user_id', as_index=False).count()[agg_column].values
#     # 不懂
#     y = df['answer'] if 'answer' in df.columns else None
#         # 不懂
#         # 不知道输出的是什么，应该带着id,带着answer
#     # s1把answer这一列删掉，删掉了成功再返回df, grp_info, y
#     # df--你输入的dataframe；grp_info-不知道是什么；y代表你的预测值-打分
#     return df.drop(columns=['answer']) if 'answer' in df.columns else df, grp_info, y

## make_recommend_dict()

调用这个函数的地方

In [ ]:
# recommend_dict = make_recommend_dict(X, y_pred)

这个函数是如何定义的

In [ ]:
def make_recommend_dict(X_val, y_pred):
    X_val['pred'] = y_pred
    _t = X_val.groupby('user_id')\
        .apply(lambda x: list(x.sort_values('pred', ascending=False)['article_id'].head(5)))\
        .reset_index()\
        .rename(columns={0: 'item_list'})

    recommend_dict = dict(zip(_t['user_id'], _t['item_list']))    
    return recommend_dict

## calc_mrr_and_hit()

调用这个代码的位置

In [ ]:
# mrr, hit, details = calc_mrr_and_hit(recommend_dict, answers)

calc_mrr_and_hit()这个函数的定义的代码

In [ ]:
# def calc_mrr_and_hit(recommend_dict, y, k=5):
#     #assert len(recommend_dict) == len(y)
#     sum_mrr = 0.0
#     sum_hit = 0.0
#     sum_hit_detail = np.repeat(0.0, 5)
#     user_cnt = len(recommend_dict.keys())

#     for user_id, recommend_items in recommend_dict.items():
#         answer = y[user_id] if user_id in y else -1
#         if (answer in recommend_items) and (recommend_items.index(answer) < k):
#             sum_hit += 1
#             sum_mrr += 1 / (recommend_items.index(answer) + 1)
#             sum_hit_detail[recommend_items.index(answer)] += 1

#     return (sum_mrr / user_cnt), (sum_hit / user_cnt), (sum_hit_detail / user_cnt)

# make_test_data()

调用这句代码的位置

In [ ]:
print("<Start># Part_【2】make_test_data");print()
make_test_data()

make_test_data()函数的定义

In [ ]:
def make_test_data():
    handler(False)

没什么好说的，这个东西就是用预处理数据并输出为csv文件

# run()

调用这句代码的位置

In [ ]:
print("<Start># Part_【3】run");print()
run()

这个run()函数的定义

就是读取处理过的数据、建模、预测、存储预测的东西-并打印

In [ ]:
# def run():
#     df_train = pd.read_csv(CACHE_FOLDER + 'train.csv')
#     df_test = pd.read_csv(CACHE_FOLDER + 'test.csv')

#     # clf = lgb.LGBMRanker(random_state=777, n_estimators=1000)
#     # !!!!!!!!!建议你将n_estimators变小一些，这样运算速度快!!!!!!!!!!!
#     # 调低 n_estimators数量
#     clf = lgb.LGBMRanker(random_state=777, n_estimators=100)

#     users = df_train['user_id'].unique()
#     train_users, eval_users = train_test_split(users, test_size=0.2, random_state=77)
#     df_new_train = df_train.merge(pd.DataFrame(train_users, columns=['user_id']))
#     df_eval = df_train.merge(pd.DataFrame(eval_users, columns=['user_id']))

#     X_train, X_grp_train, y_train = prepare_dataset(df_new_train)
#     X_eval, X_grp_eval, y_eval = prepare_dataset(df_eval)
#     X_test, X_grp_test, _ = prepare_dataset(df_test)

#     def handle_columns(X):
#         return X.drop(columns=['user_id', 'article_id'])

#     _X_train = handle_columns(X_train)

#     clf.fit(_X_train, y_train, group=X_grp_train, eval_set=[(handle_columns(X_eval), y_eval)], eval_group=[X_grp_eval], eval_at=[1, 2, 3, 4, 5], eval_metric=['ndcg', ], early_stopping_rounds=50, verbose=False)
#     print('Best iteration: {}'.format(clf.best_iteration_))
#     y_pred = clf.predict(handle_columns(X_test), group=X_grp_test, num_iteration=clf.best_iteration_)
    
#     for column, score in sorted(zip(_X_train.columns, clf.feature_importances_), key=lambda x: x[1], reverse=True):
#         print('{}: {}'.format(column, score))

#     recommend_dict = make_recommend_dict(X_test, y_pred)

#     create_submission(recommend_dict)
#     print("<finished>Part_【3】run")